<a href="https://colab.research.google.com/github/smara97/FakeNews/blob/master/quality_text_version1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from google.colab import files,drive

from nltk.corpus import stopwords
from nltk import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from torch.autograd import Variable
from string import punctuation
from gensim.models import Word2Vec

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import collections
import nltk
import torch.nn.functional as F
import matplotlib.pyplot as plt
import re
import string

nltk.download('stopwords')
nltk.download('wordnet')
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
"""
Read Dataset (Data Frame)

"""

dftrain=pd.read_csv("/content/drive/My Drive/Datasets/liar-plus/train.tsv",sep="\t",header=None)
dfval=pd.read_csv("/content/drive/My Drive/Datasets/liar-plus/val.tsv",sep="\t",header=None)
dftest=pd.read_csv("/content/drive/My Drive/Datasets/liar-plus/test.tsv",sep="\t",header=None)

In [0]:
"""
list of columns's Name 

"""

cols=['index','ID','label','statement','subject','speaker',
      'speaker_job','state','party','barely_true',
      'false','half_true','mostly_true','pants_on_fire',
      'context','justification']

In [0]:
"""
Add list cols to Data Frame columns

"""

dftrain.columns=cols
dfval.columns=cols
dftest.columns=cols

In [0]:
"""
Make Datasets have only statement,subject,justification and label 
important Feauters to Training

"""

dftrain=dftrain.loc[:,['statement','subject','justification','label']]
dfval=dfval.loc[:,['statement','subject','justification','label']]
dftest=dftest.loc[:,['statement','subject','justification','label']]
dftrain=dftrain.append(dfval)
dftrain=dftrain.append(dftest)

In [0]:
"""
Drop NAN value and index column in Datasets 

"""

dftrain=dftrain.dropna(axis=0)

dftrain=dftrain.reset_index()

dftrain=dftrain.drop(['index'],axis=1)


In [8]:
"""  Show first two's row in dataset """

dftrain.head(2)

,statement,subject,justification,label
0,Says the Annies List political group supports ...,abortion,That's a premise that he fails to back up. Ann...,false
1,When did the decline of coal start? It started...,"energy,history,job-accomplishments","Surovell said the decline of coal ""started whe...",half-true


In [0]:
"""
convert labels to 0,1 True or False  

"""
convertlabel = {
	'pants-fire': 0,
	'false': 0,
	'barely-true': 0,
	'half-true': 1,
	'mostly-true': 1,
	'true': 1
}

In [0]:
"""
Clean Text 

"""

def clean(text):
  text=text.lower()
  stp=set(stopwords.words("english"))
  placesp = re.compile('[/(){}\[\]\|@,;]')
  removech= re.compile('[^0-9a-z #+_]')
  st=WordNetLemmatizer()
  text=re.sub(placesp,' ',text)
  text=re.sub(removech,' ',text)
  text=text.split()
  text=[w for w in text if not w in stp]
  text=[st.lemmatize(w) for w in text]
  text=" ".join(text)
  text = text.translate(str.maketrans("", "", string.punctuation))
  return text

In [0]:
"""
Convert Labels dataset to 0,1 by Call convertlabel Function , 
sentence to indeces by call transfer_sent Function

"""

for i in range(dftrain.shape[0]):
  dftrain.loc[i,'label']=convertlabel[dftrain.loc[i,'label']]
  dftrain.loc[i,'statement']=clean(dftrain.loc[i,'statement'])
  dftrain.loc[i,'subject']=clean(dftrain.loc[i,'subject'])
  dftrain.loc[i,'justification']=clean(dftrain.loc[i,'justification'])

In [0]:
statements=dftrain['statement'].values
statements=[statement.split() for statement in statements]

subjects=dftrain['subject'].values
subjects=[subject.split() for subject in subjects]

justifications=dftrain['justification'].values
justifications=[justification.split() for justification in justifications]

In [0]:
user_statement=statements[0]
user_justification=justifications[0]
user_subject=subjects[0]

In [0]:
smoother = SmoothingFunction()

score_statement = sentence_bleu(statements, user_statement, smoothing_function=smoother.method2,emulate_multibleu=True)
score_subject = sentence_bleu(subjects, user_subject, smoothing_function=smoother.method2,emulate_multibleu=True)
score_justification = sentence_bleu(justifications, user_justification,smoothing_function=smoother.method2, emulate_multibleu=True)

In [17]:
all_score=(score_statement+score_subject+score_justification)/3
print(all_score)

0.8648666666666666
